In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Data preparation

In [ ]:
import pandas as pd 
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import numpy as np
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

In [ ]:
    data = pd.read_csv('../input/bank-customer-churn-modeling/Churn_Modelling.csv')
   


In [ ]:
data.head()

In [ ]:
data.columns = data.columns.str.lower()
data.columns

In [ ]:
data.columns = ['row_number', 'customer_id', 'surname', 'creditscore', 'geography',
       'gender', 'age', 'tenure', 'balance', 'num_of_products', 'has_crcard',
       'isactive_member', 'estimated_salary', 'exited']
data.columns

In [ ]:
data.info()

In [ ]:
data.shape

In [ ]:
data.describe()

In [ ]:
data['tenure'].describe()

In [ ]:
data['tenure'].hist()

We do not observe anomalies in the dataset and significant outliers. For example, the minimum age is 18 and the maximum is 92, this is normal.

In [ ]:
display(data.isna().sum())

Missing values are not observed

In [ ]:
data.duplicated().sum()

- Let's look at "Exited" :

In [ ]:
data['exited'].value_counts()

In [ ]:
print('Percentage of positive ratings: {:.2%}'.format(data['exited'].mean()))



In [ ]:
data.hist(bins=50, figsize=(20,15), edgecolor='black', linewidth=2)
plt.show()

In the process of developing functions for our model, we will remove three columns - customer_id, row_number and surname, which in our case do not carry a payload.

In [ ]:
data1 = data.drop(["row_number","customer_id","surname"],axis=1).copy()
display(data1.head())

## Take a look into this problem :

1. Let's create hists where :
- Let's look  and compare the fact of customer exit in different countries

In [ ]:
exited_yes= data1[data1['exited'] == 1]['geography']
exited_no = data1[data1['exited'] == 0]['geography']

plt.hist([exited_yes,exited_no],label=("Yes","No"))
plt.xlabel("Country")
plt.ylabel("Number of People")
plt.legend();

We can conclude that the clients of France and Germany left the bank most of all, but in France there are twice as many clients as in Germany, therefore, in percentage terms, most of the clients left in Germany.

- Let's look  and compare the fact of leaving clients with different age in the bank

In [ ]:
tenure_yes = data1[data1.exited ==1].tenure
tenure_no = data1[data1.exited ==0].tenure

plt.hist([tenure_yes, tenure_no], label = ("Yes", "No"))
plt.xlabel("Tenure")
plt.ylabel("Numper of People")
plt.legend();

We can conclude that most customers leave with a ten-year term in the bank. Unexpectedly

- Let's look and compare the fact of care for men and women

In [ ]:
sns.countplot(data=data1,x=data1['exited'], hue=data1['gender'])

We can see more women leaving the bank.

- Let's look  and compare the fact of leaving customers with their activity

In [ ]:
sns.countplot(x = data1['isactive_member'], hue = data1['exited'])

We see that inactive clients leave the most, as expected.

- Let's look and compare the fact of leaving customers with the presence of a credit card.

In [ ]:
sns.countplot(x = data1['has_crcard'], hue = data1['exited'])

It seems that the share of people with cards is approximately the same in the group of those who left and those who did not.

2. Columns of geography and gender will be converted to numeric using OHE, we need quantitative characteristics for greater accuracy

In [ ]:
geography_ohe = pd.get_dummies(data['geography'], drop_first = True)
gender_ohe = pd.get_dummies(data['gender'], drop_first=True)

data1.drop(['gender', 'geography'], axis = 1 , inplace = True)

data_ohe = pd.concat([data1, geography_ohe, gender_ohe], axis = 1)

data_ohe.head()

In [ ]:
data_ohe.info()

In [ ]:
display(data_ohe.isna().sum())

3. Let's do a correlation analysis of the data:

In [ ]:
data.corr()

In [ ]:
pd.plotting.scatter_matrix(data, figsize = (30, 30))

No correlation is observed.

4. Let's break the data into samples:

In [ ]:
features = data_ohe.drop(['exited'], axis = 1)
target = data_ohe['exited']

We get a test sample of 20% and again divide the remaining 80% to get a test sample. We will train on 60% of the data

In [ ]:
from sklearn.model_selection import train_test_split

features_train , features_remain , target_train , target_remain = train_test_split(features, target, 
                                                                                  test_size = 0.4, random_state=12345)

features_valid , features_test , target_valid , target_test = train_test_split(features_remain, target_remain,
                                                                              test_size = 0.5, random_state=12345)

In [ ]:
features_train.shape



In [ ]:
features_valid.shape

In [ ]:
features_test.shape


5. Let's do feature scaling. Scaling features across the entire dataset can lead to data leakage.
-  We will train only 'train'.

In [ ]:
from sklearn.preprocessing import StandardScaler

numeric = ['creditscore', 'age', 'balance', 'estimated_salary']
scaler = StandardScaler()
scaler.fit(features_train[numeric])

pd.options.mode.chained_assignment = None

features_train[numeric] = scaler.transform(features_train[numeric])

features_train.head()



- Let's train "valid"

In [ ]:
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_valid.head()

- train "test"

In [ ]:
features_test[numeric] = scaler.transform(features_test[numeric])
features_test.head()

6. Let's start from Logistic Regression

In [ ]:
model = LogisticRegression(random_state =12345, solver='liblinear')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print("F1:", f1_score(target_valid, predicted_valid))

- Pretty bad figure, repeat with `class_weight = 'balanced' '

In [ ]:
model = LogisticRegression(random_state =12345, solver='liblinear', class_weight = 'balanced')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print("F1:", f1_score(target_valid, predicted_valid))

- Now it is better . But now we will not choose hyperparameters, let's move on to the next algorithm

7. Random Forest

In [ ]:
model = RandomForestClassifier(random_state = 12345, n_estimators = 10)
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print("F1:", f1_score(target_valid, predicted_valid))

- Random forest showed the best result. Let's also try with class_weight = 'balanced'

In [ ]:
model = RandomForestClassifier(random_state = 12345, n_estimators = 10,
                              class_weight = 'balanced')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print("F1:", f1_score(target_valid, predicted_valid))

- We will also not choose hyperparameters now until we fix the problem with the imbalance.

### Combat imbalanced :

In [ ]:
data['exited'].value_counts().to_frame()

### Upsampling :
- Divide the training sample into negative and positive
- Duplicate the objects of the positive class and merge them with the - Objects of the negative class
- Mix data

In [ ]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    features_upsample = pd.concat([features_zeros] + [features_ones] * repeat )
    target_upsample  = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsample, target_upsample = shuffle(features_upsample, target_upsample, 
                                                random_state = 12345)
    return features_upsample, target_upsample

features_upsample, target_upsample = upsample(features_train , target_train, 5)



In [ ]:
target_upsample.shape



In [ ]:
features_upsample.shape



#### Logistic Regression :

In [ ]:
model = LogisticRegression(random_state = 12345, solver = 'liblinear')
model.fit(features_upsample, target_upsample)
predicted_valid = model.predict(features_valid)
print('F1:', f1_score(target_valid, predicted_valid))

We observe a slight increase in the metric, close to the one we received by specifying the class_weight parameter

#### Random Forest :

In [ ]:
model = RandomForestClassifier(random_state = 12345, n_estimators = 10)
model.fit(features_upsample, target_upsample)
predicted_valid = model.predict(features_valid)
print('F1:', f1_score(target_valid, predicted_valid))



There is an improvement here too.

### Dowmsampling

- divide the training sample into negative and positive objects
- randomly discard some of the negative items
- taking into account the received data, we will create a new training sample
- shuffle the data

In [ ]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    features_downsample = pd.concat(
        [features_zeros.sample(frac = fraction, random_state = 12345)] + [features_ones])
    
    target_downsample = pd.concat(
    [target_zeros.sample(frac = fraction, random_state = 12345)] + [target_ones])
    
    features_downsample, target_downsample = shuffle(features_downsample, target_downsample,
                                                    random_state = 12345)
    return features_downsample, target_downsample

features_downsample, target_downsample = downsample(features_train, target_train, 0.1)
                                   
    

In [ ]:
features_downsample.shape

In [ ]:
target_downsample.shape

#### Logistic Regression

In [ ]:
model = LogisticRegression(random_state = 12345, solver = 'liblinear')
model.fit(features_downsample, target_downsample)
predicted_valid = model.predict(features_valid)
print('F1:', f1_score(target_valid, predicted_valid))

#### Random Forest

In [ ]:
model = RandomForestClassifier(random_state = 12345, n_estimators = 10)
model.fit(features_downsample, target_downsample)
predicted_valid = model.predict(features_valid)
print('F1:', f1_score(target_valid, predicted_valid))

Downsampling shows worse results than Upsampling for all algorithms.
Let's try to change the threshold and see what the indicators will be - this time we will turn to recall and precision.

### Threshold change

#### Logistic Regression

- the threshold value is specified from 0 to 0.95 with a step of 0.02

In [ ]:
model = LogisticRegression(random_state = 12345, solver = 'liblinear')
model.fit(features_train, target_train)
probabilities_valid = model.predict_proba(features_valid) 
probabilities_one_valid = probabilities_valid[:, 1]

for threshold in np.arange(0, 0.95, 0.05):
    predicted_valid = probabilities_one_valid > threshold
    precision = precision_score(target_valid, predicted_valid)
    recall = recall_score(target_valid, predicted_valid)
    f1 = f1_score(target_valid, predicted_valid)
    print("Threshold = {:.2f} | Accuracy = {:.3f}, Completeness = {:.3f} | F1 = {:.3f}".format(
        threshold, precision, recall, f1))
    
precision, recall, threshold = precision_recall_curve(target_valid, probabilities_one_valid)
    
plt.figure(figsize=(10,10))
plt.step(recall, precision, where = 'post')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title(' Precision-Recall')
plt.show() 



For a threshold of 0, recall is 1 - all responses are positive. At a threshold of 0.85, the model stops giving correct answers. The highest value of F1 is observed at a threshold of 0.25.

#### Random Forest

In [ ]:
best_model = None
best_result = 0

for est in range(1, 20):
    model = RandomForestClassifier(random_state = 12345, n_estimators = est)
    model.fit(features_train, target_train)
    probabilities_valid = model.predict_proba(features_valid) 
    probabilities_one_valid = probabilities_valid[:, 1]
    
    for threshold in np.arange(0, 0.95, 0.05):  
        predicted_valid = probabilities_one_valid > threshold
        precision = precision_score(target_valid, predicted_valid)
        recall = recall_score(target_valid, predicted_valid)
        f1 = f1_score(target_valid, predicted_valid)
                
        if precision > best_result :
            best_model = model
            best_result = precision
        
        if recall > best_result :
            best_model = model
            best_result = recall
       
        if threshold > best_result :
            best_model = model
            best_result = threshold 
          
        if f1 > best_result :
            best_model = model
            best_result = f1
            
        print('est = {:.0f}'.format(est))
        print("Threshold = {:.2f} | Accuracy = {:.3f}, Completeness = {:.3f} | F1 = {:.3f}".format(
        threshold, precision, recall, f1))
        print('_________________________________________')
precision, recall, threshold = precision_recall_curve(target_valid, probabilities_one_valid)
    
plt.figure(figsize=(10,10))
plt.step(recall, precision, where = 'post')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('Precision-Recall')
plt.show() 

The highest score is achieved at a threshold of 0.3.

## Model testing

To test the model, we use the Random Forest model, and to combat the imbalance, we use upsampling, since it showed the best result.

In [ ]:
final_model = RandomForestClassifier(random_state = 12345, 
                                      n_estimators = 45,
                                    max_depth = 50)
final_model.fit(features_upsample, target_upsample)
prediction_test = final_model.predict(features_test)



- To assess the quality, we visualize the error matrix

In [ ]:
conf_matrix = confusion_matrix(target_test, prediction_test)
print(conf_matrix)

- Calculate metrics

In [ ]:
%%time
accuracy = accuracy_score(target_test, prediction_test)
precision = precision_score(target_test, prediction_test)
recall = recall_score(target_test, prediction_test)
f1 = f1_score(target_test, prediction_test)

print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1:', f1)

- Let's build a ROC curve

In [ ]:
probabilities = final_model.predict_proba(features_test)[:, 1] 

fpr, tpr, thresholds = roc_curve(target_test, probabilities)

plt.figure()
plt.plot(fpr, tpr)
plt.plot([0,1],[0,1], linestyle= '--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC-кривая')
plt.show()

- Calculate AUC

In [ ]:
auc_roc = roc_auc_score(target_test, probabilities)
print('AUC:',auc_roc)

# Conclusion:
Our goal was to create a model that can predict customer churn.

The main task was:

- create a model in which F1 is higher than 0.59;
- comparison of different methods of balancing target attribute classes;
- testing different models and choosing the best one.
- At the first step, research and data preprocessing were performed:
  * gaps in the Tenure column have been replaced with median values;
  * categorical features were converted to numerical ones using One-Hot Encoding
- As models for study, we used Logistic Regression and Random Forest (which performed best)
- Three methods were used to balance target attribute classes:
  * class_weight
  * upsampling
  * downsampling
- A random forest was chosen as the final model, the method of dealing with imbalance is upsampling
- An error matrix was built: True Positive (TP) - 229 True Negative (TN) - 1454 False Positive (FP) - 123 False Negative (FN) - 194
- The error matrices were calculated with the following metrics:
  * Accuracy: 0.8415
  * Precision: 0.6505681818181818
  * Recall: 0.541371158392435
  * F1: 0.590967741935484
- We built an ROC curve. AUC value - 0.8440795657433765